In [ ]:
import numpy as np
import pandas as pd

import keras 
import keras.backend as K
from keras.optimizers import Adam

from gan_utils import *
from strudel_utils import *
from networks import *

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
config.gpu_options.visible_device_list = "0"
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [ ]:

gen_shape = (1, 20736, 1)
gen_inputs, gen_encoded, gen_outputs, gen_outputs_lambda = generator(gen_shape)

dis_inputs, dis_outputs = discriminator(K.int_shape(gen_outputs)[1:])

class_intermediate_input_shapes = [(K.int_shape(gen_inputs)[1], K.int_shape(gen_inputs)[2]//(2**i), [32,32,32,32,64,64,64,64][i]) for i in range(8)]
class_in, class_inputs, class_out = classifier(K.int_shape(gen_inputs)[1:], class_intermediate_input_shapes)


In [ ]:
x, y, has_transits = load_data()
model = GAN(gen_inputs, gen_outputs, gen_encoded, Adam(0.0001, beta_1=0.5, beta_2=0.9), intersection_loss, dis_inputs, dis_outputs, class_in, class_out, dis_metrics=['accuracy'], gen_dis_loss_ratio = 0.75, gen_metrics=[dice_coef])
print('loaded GAN')

classifier = Model(class_inputs + [class_in], class_out, 'classifier')
#classifier.summary()
classifier = compile_classifier(classifier, model.encoder(), 'binary_crossentropy', Adam(), ['accuracy'], None, 'classifier', reverse_freeze=True)
print('loaded Classifier')

class_train = classifier_training(classifier, 1, 32)
log_callback = log_results(model, classifier , 100, 'scores.txt')

model.fit(x, y, gen_overtraining_multiplier=1, verbose=False, epochs=10001, batch_size=32, true_idxs=has_transits, callbacks=[class_train, log_callback])


In [ ]:
del x
del y
del has_transits
del model
del classifier


In [ ]:
params_test = np.load('./total_params_sim_test_true.npy')
SNR_test = np.load('./total_SNR_sim_test_true.npy')
x_test = np.load('./total_x_sim_test_true.npy')
columns=['t0', 'real_period', 'RpRs', 'aRs', 'b', 'a0', 'mean', 'std', 'magnitude', 'SNR']
for i in np.random.randint(0, x_test.shape[0],100):
    listv = params_test[i].tolist()+[SNR_test[i,0]]
    print i, {columns[idx]:item for idx, item in enumerate(listv)}
    fig = plt.figure(figsize=(15,10))
    plt.scatter(np.arange(0,20610),x_test[i,:20610,0], s=0.5)
    plt.show()
    print(" ")


In [ ]:
x, y, params = load_data_now()
model = GAN(gen_inputs, gen_outputs, Adam(0.0001, beta_1=0.5, beta_2=0.9), intersection_loss, dis_inputs, dis_outputs, class_in, class_out, dis_metrics=['accuracy'], gen_dis_loss_ratio = 0.9, gen_metrics=[dice_coef])
generator_model = model.gen_dis()
print('loaded GAN')

classifier = Model(class_inputs + [class_in], class_out, 'classifier')
classifier_model = compile_classifier(classifier, model.encoder(), 'binary_crossentropy', Adam(), ['accuracy'], None, 'classifier', reverse_freeze=True)
print('loaded Classifier')


print('loading models')
generator_model.load_weights('best_generator_combined_test.h5')
classifier_model.load_weights('best_classifier_combined_test.h5')
print('loaded models')

y_test = np.load('./total_params_sim_test.npy')[:,1] > 0
x_test = np.expand_dims(np.load('./total_x_sim_test.npy', mmap_mode='r'),axis=1)
x_test = np.pad(x_test, ((0,0), (0,0) ,(0, 30+ 96), (0,0)), 'constant', constant_values=(0, 0))
y_pred = classifier_model.predict(x_test, verbose=1)[:,0]
fpr, tpr, thresholds_keras = roc_curve(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
thresh = thresholds_keras[find_nearest(fpr, 0.01)]
rate = tpr[find_nearest(fpr, 0.01)]

mags = params[:,-1]
SNR = np.load('./total_SNR_sim.npy', mmap_mode='r')
print("Finished Loading Data")

print(thresh, rate, roc_auc)
pred_params = pd.DataFrame(None, columns=['t0', 'real_period', 'RpRs', 'aRs', 'b', 'a0', 'mean', 'std', 'magnitude', 'predicted_period', 'SNR','classification'])



In [ ]:
generator_out  = np.zeros((x.shape[0]-20000,20610), dtype=np.float32)
real_classifier_predictions = params[:, 1] > 0
real_classifier_predictions = real_classifier_predictions[20000:].astype(dtype=bool)
classifier_predictions = np.zeros((x.shape[0]-20000,1), dtype=np.bool)
pred_params = np.zeros((x.shape[0]-20000,13), dtype=np.float32)
for i in tqdm(xrange(20000,x.shape[0])):
    classifier_out = classifier_model.predict(x[i:i+1])[:,0]
    temp_p = params[i].tolist()
    if params[i,1] > 0:
        gen_out = generator_model.predict(x[i:i+1])[0]
        gen_out = gen_out[:, 0, :20610, 0]
        generator_out[i-20000, :] = gen_out
        temp_p.append(np.log10(process_transit([gen_out[0], params[i,1],1000])[0]))
    else:
        temp_p.append(-1)
    temp_p.append(SNR[i])
    temp_p.append(classifier_out[0])
    temp_p.append(0)
    #print dict_p
    temp = np.array(temp_p, dtype=np.float32)
    pred_params[i-20000, :] = temp
    #print pred_params
        
    classifier_predictions[i-20000,0] = bool(classifier_out[0])


In [ ]:
columns='t0, real_period, RpRs, aRs, b, a0, mean, std, magnitude, predicted_period, SNR, classification,dice coeff'
np.save("transit_parameters.npy", pred_params)
np.savetxt("transit_parameters.csv", pred_params, delimiter=",", header=columns)

In [ ]:
np.save("predicted_transits.npy", generator_out)

print('DONE Precidted Transits')

np.save("real_classification.npy", real_classifier_predictions)
np.savetxt("real_classification.csv", real_classifier_predictions, delimiter=",")
print('DONE real classification')

np.save("predicted_classification.npy", classifier_predictions)
np.savetxt("predicted_classification.csv", classifier_predictions, delimiter=",")
print('DONE classification')



x = np.load('./total_x_sim.npy', mmap_mode='r')
x = x[20000:, :, 0]
print('X loaded')
np.save("real_data.npy", x)

del x
print('DONE Real X')

y = np.load('./total_transits_sim.npy')
y = y[20000:, :, 0]
y_norm = np.max(y, axis=1, keepdims=True)
y_norm[y_norm[:, 0] == 0, 0] = np.min(y_norm[y_norm[:, 0] > 0, 0])
y /= y_norm

print('Y loaded')
np.save("real_transits.npy", y)

del y
print('DONE Real  Transits')


In [ ]:
x = np.expand_dims(np.load('./total_x_sim.npy', mmap_mode='r'),axis=1)
classifier_out = None
classifier_predictions = None

print("Starting")
for i in tqdm(xrange(20000,x.shape[0])):
    temp_x = x[i:i+1]
    temp_x = np.pad(temp_x, ((0, 0), (0, 0), (0, 30 + 96), (0, 0)), 'constant', constant_values=(0, 0))
    classifier_out = classifier_model.predict(temp_x)[:,0]
    if i == 20000: classifier_predictions = classifier_out 
    else: classifier_predictions = np.concatenate((classifier_predictions, classifier_out), axis=0)

classifier_real = np.load('./total_params_sim.npy')[:,1] > 0
fpr, tpr, thresholds_keras = roc_curve(classifier_real[20000:], classifier_predictions)
auc = roc_auc_score(classifier_real[20000:], classifier_predictions)

plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Keras (area = {:.5f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlim(0, 0.02)
plt.show()


print('Percentage at 0.001: %.5f, Percentage at 0.01: %.5f, Class AUC: %.5f\n' % (tpr[find_nearest(fpr, 0.001)], tpr[find_nearest(fpr, 0.01)], auc))

d = {'tpr': tpr, 'fpr': fpr, 'thresholds': thresholds_keras}
roc = pd.DataFrame(data=d)
roc.to_csv('ROC_AUC_FULL.csv')




In [ ]:
real_tranists_y = np.load("real_transits.npy", mmap_mode='r')
generator_out = np.load("predicted_transits.npy", mmap_mode='r')
transit_parameters = np.loadtxt("transit_parameters.csv", delimiter=',')

x = np.load('./total_x_sim.npy', mmap_mode='r')
x = x[20000:, :, 0]

dice_coeffs = np.zeros([x.shape[0]])
dice_coeffs = iterate_efficiently(real_tranists_y, generator_out, dice_coeffs, 20)

transit_parameters[:,-1] = 0
transit_parameters = transit_parameters[len(transit_parameters) - real_tranists_y.shape[0]:]
transit_parameters[:,-1] = dice_coeffs
transit_parameters = transit_parameters[transit_parameters[:, 1] != 0]

columns='t0, real_period, RpRs, aRs, b, a0, mean, std, magnitude, predicted_period, SNR, classification, dice coeff'
np.save("transit_parameters.npy", transit_parameters)
np.savetxt("transit_parameters.csv", transit_parameters, delimiter=",", header=columns)
